In [102]:
import numpy as np
import pandas as pd

print(np.__version__)
print(pd.__version__)

1.18.5
1.0.5


###1. Take the Teams and Games CSV files and load them into their own DataFrames.

In [103]:
games = pd.read_csv('Games.csv', names=['Date', 'TeamH', 'ScoreH', 'TeamA', 'ScoreA'])

In [ ]:
games.head(8)

,Date,TeamH,ScoreH,TeamA,ScoreA
0,13/11/2015,Hawaii,87,Montana St.,76
1,13/11/2015,Eastern Mich.,70,Vermont,50
2,13/11/2015,Columbia,107,Kean,62
3,13/11/2015,La.-Monroe,88,McMurry,43
4,13/11/2015,Yale,70,Fairfield,57
5,13/11/2015,North Dakota,99,Minn.-Morris,69
6,13/11/2015,Central Mich.,89,Jacksonville St.,83
7,13/11/2015,Army,93,Ferrum,54


In [104]:
teams = pd.read_csv('Teams.csv', names=['Region','Team'])

In [ ]:
teams.head(6)

,Region,Team
0,America East,Vermont
1,America East,Stony Brook
2,America East,UMBC
3,America East,Hartford
4,America East,Albany
5,America East,Massachusetts-Lowell


### 2. Clean the data: 

In [106]:
# ensure that all of the team names match between the Teams and Games DataFrames
print(teams[teams.Team.str.startswith('St ')].Team)
print(teams[teams.Team.str.endswith(' State')].Team)

games.TeamH.replace(' St.', ' State', inplace=True)
games.TeamA.replace(' St.', ' State', inplace=True)

24       St Bonaventure
71            St John's
176          St Peter's
223     St Francis (PA)
227    St Francis (BKN)
Name: Team, dtype: object
14            Wichita State
38            Florida State
43     North Carolina State
58           Kennesaw State
60             Kansas State
               ...         
324           Georgia State
330          Arkansas State
331       Appalachian State
344        New Mexico State
352           Chicago State
Name: Team, Length: 64, dtype: object


In [107]:
mask = ['State' in name[-5:] for name in games.TeamH.unique()]
games_StateH = games.TeamH.unique()[mask]
print(len(games_StateH))

mask = ['State' in name[-5:] for name in games.TeamA.unique()]
games_StateA = games.TeamA.unique()[mask]
print(len(games_StateA))

39
49


In [108]:
# only teams (from teams.Teams) can play each other at any given date
df = games.loc[(games.TeamH.isin(teams.Team)) & (games.TeamA.isin(teams.Team))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9300 entries, 4 to 23492
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    9300 non-null   object
 1   TeamH   9300 non-null   object
 2   ScoreH  9300 non-null   int64 
 3   TeamA   9300 non-null   object
 4   ScoreA  9300 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 435.9+ KB


In [109]:
# check for both zero score entries
print(len(df[(df.ScoreH == 0) & (df.ScoreA == 0)]))

# remove them
df = df.drop(df[(df.ScoreH == 0) & (df.ScoreA == 0)].index)

79


In [110]:
print(df.TeamH.nunique())
print(df.TeamA.nunique())
df.info()

263
263
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9221 entries, 4 to 23492
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    9221 non-null   object
 1   TeamH   9221 non-null   object
 2   ScoreH  9221 non-null   int64 
 3   TeamA   9221 non-null   object
 4   ScoreA  9221 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 432.2+ KB


### 3. Prepare the data for processing by creating columns that we can train against.

In [111]:
df_mod = df.copy()

# Are there any draws?
print(len(df_mod[df_mod.ScoreH == df_mod.ScoreA]))

0


In [112]:
# Home wins and away wins 
df_mod['WinH'] = 0
df_mod.loc[df_mod.ScoreH > df_mod.ScoreA, 'WinH'] = 1
df_mod['WinA'] = 0
df_mod.loc[df_mod.ScoreA > df_mod.ScoreH, 'WinA'] = 1

In [114]:
# analyse home game results
dfH = pd.DataFrame(columns=['team', 'playedH', 'totalH', 'totalA', 'diffH', 'totWinH', 'avgH'])
i = 0
for team in np.unique(df_mod.TeamH):
    tgH = df_mod.loc[df_mod.TeamH == team, ['ScoreH', 'ScoreA', 'WinH']]
    playedH = len(tgH)
    totalH = tgH.ScoreH.sum()
    totalA = tgH.ScoreA.sum()
    totWinH = tgH.WinH.sum()
    diffH = totalH - totalA
    avgH = round(diffH/playedH, 2)
    dfH.loc[i] = [team, playedH, totalH, totalA, diffH, totWinH, avgH]
    i += 1

In [115]:
# analyse away game results
dfA = pd.DataFrame(columns=['team', 'playedA', 'totA', 'totH', 'diffA', 'totWinA', 'avgA'])
i = 0
for team in np.unique(df_mod.TeamA):
    tgA = df_mod.loc[df_mod.TeamA == team, ['ScoreH', 'ScoreA', 'WinA' ]]
    playedA = len(tgA)
    totA = tgA.ScoreA.sum()
    totH = tgA.ScoreH.sum()
    totWinA = tgA.WinA.sum()
    diffA = totA - totH
    avgA = round(diffA/playedA, 2)
    dfA.loc[i] = [team, playedA, totA, totH, diffA, totWinA, avgA]
    i += 1

In [116]:
# combine each team's home and away game stats
dfHA = dfH.merge(dfA, how='inner', on='team')
# list top 10
dfHA.sort_values(by=['totWinH', 'avgH', 'totWinA', 'avgA'], ascending=False).head(10)

,team,playedH,totalH,totalA,diffH,totWinH,avgH,playedA,totA,totH,diffA,totWinA,avgA
246,Villanova,67,5324,4341,983,61,14.67,45,3506,3162,344,33,7.64
105,Kansas,62,5066,4233,833,58,13.44,35,2616,2564,52,23,1.49
78,Gonzaga,60,5185,3843,1342,55,22.37,41,3484,2833,651,37,15.88
154,North Carolina,65,5505,4581,924,55,14.22,42,3330,3058,272,28,6.48
108,Kentucky,61,4994,4128,866,53,14.20,40,2998,2905,93,22,2.33
247,Virginia,59,4136,3267,869,52,14.73,47,3139,2742,397,33,8.45
56,Duke,60,4969,3954,1015,51,16.92,48,3786,3508,278,31,5.79
178,Purdue,57,4606,3737,869,49,15.25,41,3004,2823,181,24,4.41
133,Michigan,55,4012,3322,690,49,12.55,45,3214,3120,94,25,2.09
261,Xavier,62,4993,4487,506,49,8.16,48,3490,3530,-40,25,-0.83


### 4. Save the output from your added columns to Games-Calculated.csv


In [117]:
dfHA.to_csv('Games-Calculated.csv')